In [ ]:
from __init__ import *

In [ ]:
directory_path = "../output/signal/"

kraft.path.path(directory_path)

In [ ]:
def get_surprise(r, d, _0=kraft.CONSTANT.FLOAT_RESOLUTION):

    rp = _0

    r_sum = r.sum()

    if 0 < r_sum:

        rp += r / r_sum

    dp = _0

    d_sum = d.sum()

    if 0 < d_sum:

        dp += d / d_sum

    return dp * np.log(dp / rp)


def update_surprise(r, dataframe):

    for vector in dataframe.to_numpy():

        surprises = get_surprise(r, vector)

        for i in range(vector.size):

            vector[i] = surprises[i]

    kraft.feature_x_sample.summarize(dataframe)

In [ ]:
for i, dict_ in enumerate(SETTING["model_data_dicts"]):

    print(dict_["name"])

    dataframe = pd.read_csv(dict_["file_path"], sep="\t", index_col=0)

    dataframe.columns.name = dict_["axis_1_name"]

    update_surprise(dataframe.sum(), dataframe)

    negative = dataframe.clip(upper=0)

    positive = dataframe.clip(lower=0)

    negative = negative.loc[
        kraft.series.get_extreme_labels(
            negative.median(axis=1), "<", standard_deviation=1.5
        ),
        :,
    ]

    positive = positive.loc[
        kraft.series.get_extreme_labels(
            positive.median(axis=1), ">", standard_deviation=2
        ),
        :,
    ]

    negative.index = ("(-) {}".format(label) for label in negative.index.to_numpy())

    positive.index = ("(+) {}".format(label) for label in positive.index.to_numpy())

    signal = pd.concat((-negative, positive))

    signal.index.name = dataframe.index.name

    signal.to_csv("{}/{}.tsv".format(directory_path, i), sep="\t")